In [ ]:
import os,sys
import mne
import utils  #my code
import json
import matplotlib.pyplot as plt
import numpy as np
import h5py
%load_ext autoreload
%autoreload 2

if os.environ.get('DATA_DUSS') is not None:
    data_dir = os.path.expandvars('$DATA_DUSS')
else:
    data_dir = '/home/demitau/data'
    


rawname_ = 'S01_off_hold'
#rawname_ = 'S01_on_hold'
rawname = rawname_ + '_resample_raw.fif'
fname_full = os.path.join(data_dir,rawname)
    
# read file -- resampled to 256 Hz,  Electa MEG, EMG, LFP, EOG channels
raw = mne.io.read_raw_fif(fname_full, None)

In [ ]:
reconst_name = rawname_ + '_resample_afterICA_raw.fif'
reconst_fname_full = os.path.join(data_dir,reconst_name)
reconst_raw = mne.io.read_raw_fif(reconst_fname_full, None)

In [ ]:
#src_fname

In [ ]:
'srcd_S01_off_hold_HirschPt2011_test.mat'

In [ ]:
src_fname = 'srcd_{}_HirschPt2011_test.mat'.format(rawname_)
src_fname_full = os.path.join(data_dir,src_fname)
src_ft = h5py.File(src_fname_full, 'r')
ff = src_ft

In [ ]:
ff.keys()

In [ ]:
ff['source_data']

In [ ]:
ff['source_data'][0,0]

In [ ]:
f = ff[ ff['source_data'][0,0] ]
f.keys()

In [ ]:
''.join( map(chr, f['source_data']['method'][:,0] ) )

In [ ]:
f['source_data']['avg'].keys()

# Load LCMV

In [ ]:
reconst_raw.load_data()

In [ ]:
#ff['source_data'].shape[0]

In [ ]:
sind_str,medcond,task = utils.getParamsFromRawname(rawname)

stcs = []
fbands = []
custom_raw = None
bandnames = ['tremor', 'beta', 'gamma', 'allf']
for srcdi in range(ff['source_data'].shape[0] ):
    bandname = bandnames[srcdi]
    f = ff[ ff['source_data'][srcdi,0] ]

    freqBand = f['bpfreq'][:].flatten()
    pos = f['source_data']['pos'][:,:].T
    # first coord is the left-right coord
    posinds = np.argsort( pos[:,0] )  # I'll need to sort correspondance as well

    sortedPos = pos[posinds]
    #leftHemispherePos = sortedPos[sortedPos[:,0]<= 0]
    #rightHemispherePos = sortedPos[sortedPos[:,0] > 0]
    leftInds = np.where(sortedPos[:,0]<= 0)[0]
    rightInds = np.where(sortedPos[:,0] > 0)[0]
    vertices = [leftInds, rightInds]

    t0 = f['source_data']['time'][0,0]
    tstep = np.diff( f['source_data']['time'][:10,0] ) [0]

    srcRefs = f['source_data']['avg']['mom'][0,:] 
    srcData_ = [0]* len(srcRefs)
    for srci in range(len(srcRefs)):
        srcData_[srci] = f[srcRefs[srci] ][:,0]

    srcData = np.vstack(srcData_)[posinds]

    #(data, vertices=None, tmin=None, tstep=None, subject=None, verbose=None
    stc = mne.SourceEstimate(data = srcData, tmin = t0, tstep= tstep  , subject = sind_str , vertices=vertices)
    
    stcs += [stc]
    fbands += [freqBand]
    
    lhi = map(str, list( vertices[0] ) )
    rhi = map(str, list( vertices[1] ) )

    srcnames = [ 'srcL_{}_'.format(bandname) + s for s in lhi ] 
    srcnames += [ 'srcR_{}_'.format(bandname) + s for s in rhi ]

    # Initialize an info structure
    info = mne.create_info(
        ch_names=srcnames,
        ch_types=['csd'] * len(srcnames),
        sfreq=int ( 1/tstep ))

    custom_raw_cur = mne.io.RawArray(srcData, info)
    if custom_raw is None:
        custom_raw = custom_raw_cur
    else:
        custom_raw.add_channels([custom_raw_cur])
    #print(custom_raw)
    #reconst_raw.add_channels([custom_raw])
    

# Average sources

In [ ]:
import scipy.io

srcCoords_fn = sind_str + '_modcoord.mat'
crdf = scipy.io.loadmat(srcCoords_fn)
coords = crdf['coords_Jan_actual']
srcgroups = crdf['point_ind_corresp'][0][posinds]-1

In [ ]:
assert min(srcgroups) == 0
assert max(srcgroups) == len(coords)-1, ( max(srcgroups)+1, len(coords) )
newchanames = []
newdatas = []
for i in range( max(srcgroups)+1 ):
    inds = np.where(srcgroups == i)[0]
    #srcData[inds]
    #L or R?
    if coords[i][0] <= 0:
        brainside = 'L'
    else:
        brainside = 'R'
    
    for bandname in bandnames:
        chnames = [ 'src{}_{}_{}'.format(brainside,bandname,s) for s in inds ] 
        chdata, times = custom_raw[chnames]
        newdata = np.mean(chdata,axis=0)[None,:]
        #print(chnames)
        print('{}: Mean over {}'.format(i,chdata.shape[0]))
        newchname = 'msrc{}_{}_{}'.format(brainside,bandname,i)
        newchanames += [newchname]
        newdatas    += [newdata]

In [ ]:
info = mne.create_info(
    ch_names=newchanames,
    ch_types=['csd'] * len(newchanames),
    sfreq=int ( 1/tstep ))

custom_raw.add_channels([ mne.io.RawArray(np.vstack(newdatas), info)] )

 ## Plotting

In [ ]:
meansrcind = 1
inds = np.where(srcgroups == meansrcind)[0]
bandname = 'gamma'
if coords[meansrcind][0] <= 0:
    brainside = 'L'
else:
    brainside = 'R'
formatstr = 'src{}_{}_{}'
chns = [formatstr.format(brainside,bandname,i) for i in inds]   #msrcR_tremor_0
chns += ['m'+formatstr.format(brainside,bandname,meansrcind) ]
chns

In [ ]:
raw_compareAv = custom_raw.copy()
raw_compareAv.pick_channels(chns)

In [ ]:
%matplotlib qt
raw_compareAv.plot();

In [ ]:
#mne.time_frequency.csd_array_morlet(picks )

In [ ]:
#reconst_raw.get_channel_types()

In [ ]:
import pandas as pd

In [ ]:
td.shape

In [ ]:
vr = pd.Series(td).rolling(100).var()
vr

In [ ]:
help( pd.Series(td).rolling )

## Hjorth

In [ ]:
help( np.diff )

In [ ]:
yty = np.arange(10)[None,:]
ytyd = np.diff(yty,axis=-1, prepend=yty[:,0][:,None]  )
yty.shape, ytyd.shape

In [ ]:
def H_difactmob(dat,dt, windowsz = None):
    dif = np.diff(dat,axis=-1, prepend=dat[:,0][:,None] ) / dt
    if windowsz is None:    
        activity = np.var(dat, axis=-1)
        vardif = np.var(dif, axis=-1)
    else:
        #raise ValueError('not implemented yet')
        if dat.ndim > 1 and dat.shape[0] == 1:            
            activity = []
            vardif = []
            for dim in range(dat.shape[0]):
                act = pd.Series(dat[dim]).rolling(windowsz).var()
                var   = pd.Series(dif[dim]).rolling(windowsz).var()  # there is one-bin shift here, better to remove..
                activity += [act]
                vardif += [var]
            activity = np.vstack(activity)
            vardif = np.vstack(vardif)
        else:
            raise ValueError('wrong!')
        
    mobility = np.sqrt( vardif / activity )

    return dif,activity, mobility

def Hjorth(dat, dt, windowsz = None):
#     if windowsz is not None:
#         raise ValueError('not implemented yet')
#     activity = np.var(dat, axis=-1)
#     dif = np.diff(dat,axis=-1) / dt
#     vardif = np.var(dif)
#     mobility = np.sqrt( vardif / activity )
    dif, activity, mobility = H_difactmob(dat,dt, windowsz=windowsz)
    #dif2 = np.diff(dif) / dt
    
    dif2, act2, mob2 = H_difactmob(dif,dt, windowsz=windowsz)
    complexity = mob2 / mobility
    
    return activity, mobility, complexity

In [ ]:
srcdata_test.shape

In [ ]:
activity, mobility, complexity = Hjorth(srcdata_test, times[1]-times[0] , windowsz=256)
activity, mobility, complexity

In [ ]:
plt.subplot(3,1,1).plot(times,activity.T)
plt.subplot(3,1,1).set_title('activity')
plt.subplot(3,1,2).plot(times,mobility.T)
plt.subplot(3,1,2).set_title('mobility')
plt.subplot(3,1,3).plot(times,complexity.T)
plt.subplot(3,1,3).set_title('complexity')

In [ ]:
activity, mobility, complexity = Hjorth(srcdata_test, times[1]-times[0])
activity, mobility, complexity

In [ ]:
tt = np.arange(1,1000,0.01)
td = np.sin(tt)

In [ ]:
Hjorth(td,tt[1]-tt[0])

In [ ]:
help(plt.psd)

In [ ]:
v,f = plt.psd(td,Fs=1/(tt[1]-tt[0]) )

In [ ]:
np.sum(v) * (f[1]-f[0])

In [ ]:
srcdata_test,times = custom_raw['msrcL_gamma_1']

In [ ]:
srcdata_test.shape

In [ ]:
raw.times

In [ ]:
custom_raw.info

In [ ]:
msrcOnly = custom_raw.copy()

In [ ]:
picks =  mne.pick_channels_regexp(msrcOnly.ch_names, 'msrc._all_*'  )  
picks_names = [msrcOnly.ch_names[i] for i in picks]

In [ ]:
picks_names

In [ ]:
msrcOnly.pick_channels(   picks_names  )

In [ ]:
strec = 0
endrec = msrcOnly.times[-1]
epdur = endrec
events_one = mne.make_fixed_length_events(msrcOnly, start=strec, stop=endrec, duration=epdur)
epochs_one = mne.Epochs(msrcOnly,events_one, tmin=0,tmax = epdur, baseline=None)

In [ ]:
#tfr_array_morlet
freqs = np.arange(4,100)
tfrres = mne.time_frequency.tfr_morlet(epochs_one, freqs, freqs * 0.75, return_itc=0, n_jobs=10, 
                                       output='complex', average=False)

In [ ]:
tfrres.ch_names

In [ ]:
tfrres.data.shape

In [ ]:
#help(mne.time_frequency.tfr_morlet)

In [ ]:
windowssz = 1 #sec
window_centers = np.arange()

In [ ]:
#tfrres.data

In [ ]:
#srcData.shape

In [ ]:
dat = tfrres.data[0]
n_channels = dat.shape[0]
csds = []
for chn in range(n_channels):
    #r = dat[[chn]] * np.conj ( dat[chn:] )    # upper diagonal elements only, same freq cross-channels
    r = np.conj ( dat[[chn]] ) *  ( dat[chn:] )    # upper diagonal elements only, same freq cross-channels
    print(r.shape)
    csds += [r / custom_raw.info['sfreq'] ]

In [ ]:
#csds[0][0] - csds[0][1]

In [ ]:
csd = np.vstack( csds )

In [ ]:
def getCsdVals(csd,i,j, updiag = 0):
    #swap
    assert i < n_channels
    assert j < n_channels
    
    if j < i: 
        tmp = i
        i = j
        j = tmp

    idx = 0
    for ii in range(i):
        idx += (n_channels - ii - updiag)

    dist_to_diag = j - i - updiag
    idx += dist_to_diag
    #print(idx)
    return csd[idx]
#i,j -> 
getCsdVals(csd,2,2)

In [ ]:
#getCsdVals(csd,3,3)

In [ ]:
#getCsdVals(csd,2,2) - getCsdVals(csd,2,5)

In [ ]:
getCsdVals(csd,2,2).shape

In [ ]:
def getFullMat(csd, freq_index, time_index, updiag=0):
    M = np.zeros( (n_channels, n_channels), dtype = csd.dtype)
    for i in range(n_channels):
        for j in range(n_channels):
            M[i,j] = getCsdVals(csd,i,j, updiag =updiag)[freq_index,time_index]
            
    return M
M = getFullMat(csd, 0, 1000)

In [ ]:
#csd[1]

In [ ]:
plt.imshow(np.abs(M) )

In [ ]:
import gc; gc.collect()

In [ ]:
import scipy.signal as sig

In [ ]:
epdat = epochs_one.get_data()[0]

In [ ]:
fqs, csdscipy = sig.csd(epdat[0],epdat[3],fs=int(raw.info['sfreq']))

In [ ]:
freqs[2]

In [ ]:
fqs[6]

In [ ]:
csdscipy[6]

In [ ]:
np.mean( getCsdVals(csd,0,3)[2] )

In [ ]:
# Plot CSD
ii,jj = 0,3
plt.semilogy(fqs,csdscipy.real, label='scipy' )
plt.semilogy(freqs, np.mean( getCsdVals(csd,ii,jj), axis=1 ).real , label='hand' )
plt.legend()

In [ ]:
# Plot CSD
plt.semilogy(fqs,csdscipy.imag, label='scipy' )
plt.semilogy(freqs, np.mean( getCsdVals(csd,ii,jj), axis=1 ).imag , label='hand' )
plt.legend()

In [ ]:
cohs = []
ps = []
for i in range(n_channels):
    for j in range(i+1,n_channels):
        if i == j:
            coh_cur = np.ones( (csd.shape[1], csd.shape[2]) )
        else:
            csd_cur = getCsdVals(csd,i,j)
            csd_cur_sq = np.square(csd_cur.imag) + np.square(csd_cur.real)
            coh_cur = csd_cur_sq / (getCsdVals(csd,i,i).real * getCsdVals(csd,j,j).real  )
        ps +=  [(i,j)]
            #print('--------------')
            #display(coh_cur)
        cohs += [coh_cur[None,:].real ]

In [ ]:
coh_cur.shape

In [ ]:
fqs,cohscipy = sig.coherence(epdat[ii],epdat[jj],fs=int(raw.info['sfreq']) )

csd_cur = getCsdVals(csd,ii,jj)
csd_cur_sq = np.square(csd_cur.imag) + np.square(csd_cur.real)
coh_cur = csd_cur_sq / ( getCsdVals(csd,ii,ii) * getCsdVals(csd,jj,jj) )
coh_cur_av = np.mean(coh_cur, axis=1)

#plt.semilogy(freqs, np.mean( getCsdVals(csd,0,3), axis=1 ).imag , label='hand' )

plt.semilogy(fqs,cohscipy, label='scipy')
plt.semilogy(freqs,coh_cur_av, label='hand')
plt.legend()

In [ ]:
yy = np.mean(csd_cur, axis=1)
yy = yy.imag**2 + yy.real**2
yy1 = np.mean(getCsdVals(csd,ii,ii), axis=1  )
yy2 = np.mean(getCsdVals(csd,jj,jj), axis=1  )

In [ ]:
plt.semilogy( freqs, yy/(yy1*yy2 ) )  

plt.semilogy(fqs,cohscipy, label='coh scipy')
plt.legend()

In [ ]:
plt.semilogy( freqs, np.mean(csd_cur_sq, axis=1) )
# plt.semilogy( freqs, np.mean(getCsdVals(csd,ii,ii).real, axis=1)**2 )
# plt.semilogy( freqs, np.mean(getCsdVals(csd,jj,jj).real, axis=1)**2 )

plt.semilogy( freqs, np.mean(getCsdVals(csd,jj,jj).real * getCsdVals(csd,ii,ii).real, axis=1), ls='--' )

In [ ]:
plt.semilogy( freqs, np.mean(getCsdVals(csd,ii,ii), axis=1) )

In [ ]:
cohs[2][0,:,2000]  #all freq

In [ ]:
ps

In [ ]:
len(cohs)

In [ ]:
cohs[0].shape

In [ ]:
cohs[1].shape

In [ ]:
coh = np.concatenate(cohs,axis=0)

In [ ]:
coh.shape

In [ ]:
coh

In [ ]:
#coh(i,j,freq) = np.abs(  csd(i,j,freq) )**2 / ( csd(i,i,freq) * csd(j,j,freq))

In [ ]:
#[np.mean(psds[[i]] * psds_conj[i:], axis=2)       for i in range(n_channels)]

#     # Scaling by sampling frequency for compatibility with Matlab
#     csds /= sfreq

In [ ]:
# MNE:   _csd_morlet(...)
#     n_channels = data.shape[0]
#     csds = np.vstack([np.mean(psds[[i]] * psds_conj[i:], axis=2)
#                       for i in range(n_channels)])

#     # Scaling by sampling frequency for compatibility with Matlab
#     csds /= sfreq

 ## mne.SourceEstimate

data
    
    array of shape (n_dipoles, n_times) | tuple, shape (2,)

    The data in source space. When it is a single array, the left hemisphere is stored in data[:len(vertices[0])] and the right hemisphere is stored in data[-len(vertices[1]):]. When data is a tuple, it contains two arrays:

        “kernel” shape (n_vertices, n_sensors) and

        “sens_data” shape (n_sensors, n_times).

    In this case, the source space data corresponds to np.dot(kernel, sens_data).
vertices

    list of array, shape (2,)

    Vertex numbers corresponding to the data. The first element of the list contains vertices of left hemisphere and the second element contains vertices of right hemisphere.


# DICS

In [ ]:
f['source_data']['avg']['noise'][0,:]

In [ ]:
f['source_data']['avg']['pow'][0,:]

In [ ]:
f['source_data']['freq']

In [ ]:
f = src_ft
times = f['source_data']['time'][ts:te,0]

src = f['source_data'] 

In [ ]:
ref = src['avg']['mom'] [0,srci]
# maybe I can make more efficient data extraction of multiple channels at once 
# later if needed, if I bundle all channels from the same source file 
if f[ref].size > 10:
    srcval = f[ref ][ts:te,0]   # 1D array with reconstructed source activity

In [ ]:
freqdat_fname_full =  os.path.join(data_dir, 'tmp_freq_data.mat' )

In [ ]:
freqd = h5py.File(freqdat_fname_full, 'r')

In [ ]:
freqd['#refs#']

In [ ]:
freqd['freqdata'].keys()

In [ ]:
freqd['freqdata']['grad'].keys()

In [ ]:
freqd['freqdata']['freq'][:,0]

In [ ]:
freqd['freqdata']['labelcmb']

In [ ]:
fr = freqd[ freqd['freqdata']['labelcmb'][1,0] ]

In [ ]:
''.join( map(chr, fr[:,0] ) )

In [ ]:
freqd['freqdata']['powspctrm']

In [ ]:
freqd['freqdata']['crsspctrm']

In [ ]:
freqd['freqdata']['dimord'][:,0]

In [ ]:
''.join( map(chr, freqd['freqdata']['dimord'][:,0] ) )

In [ ]:
freqd['freqdata']['crsspctrm'][0,1]